<a href="https://colab.research.google.com/github/Viswas-Reddy-PallamReddy/DeltaTrace/blob/main/DeltaTrace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DeltaTrace :** (Versioned Parquet Store)

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -q psutil tqdm

In [20]:
from pathlib import Path

PROJECT_FOLDER = Path('/content/drive/MyDrive/DeltaTrace')
DATASET1 = PROJECT_FOLDER/'yellow_tripdata_2025-01.parquet'
DATASET2 = PROJECT_FOLDER/'yellow_tripdata_2025-02.parquet'

In [21]:
def create_folder_structure(root : str) :
    root = Path(root)
    for sub in ("base","versions", "snapshots", "logs"):
        (root / sub).mkdir(parents = True, exist_ok=True)

    history_path = root / "logs" / "history.json"
    if not history_path.exists():
        history_path.write_text("[]")
    print(f"✅ Delta Trace root ready at {root.resolve()}")

In [22]:
import uuid
import pandas as pd

def ensure_column(df : pd.DataFrame, column : str = "_row_id") -> pd.DataFrame:
    if column not in df.columns:
        df[column] = pd.Series((uuid.uuid4().hex for _ in range(len(df))), index=df.index)
    else:
        missing = df[column].isna() | (df[column] == "")
        missing_count = missing.sum()
        if missing_count > 0:
            df.loc[missing, column] = pd.Series((uuid.uuid4().hex for _ in range(missing_count)), index=df[missing].index)
        if df[column].duplicated().any():
          raise ValueError(f"❌ Duplicate '{column}' values found!")

    return df

## Row Level Diffs

In [23]:
import pandas as pd

def appended_deleted_rows(df1: pd.DataFrame, df2 : pd.DataFrame):
    df1_ids = set(df1["_row_id"])
    df2_ids = set(df2["_row_id"])

    appended_ids = df2_ids - df1_ids
    deleted_ids = df1_ids - df2_ids
    unchanged_ids = df1_ids & df2_ids

    return {
        "appended" : appended_ids,
        "deleted" : deleted_ids,
        "unchanged" : unchanged_ids
    }

In [24]:
import pandas as pd
from typing import Tuple, List, Dict, Optional, Iterable

def detect_updates(
    df_old: pd.DataFrame,
    df_new: pd.DataFrame,
    *,
    id_col: str = "_row_id",
    columns: Optional[List[str]] = None,
    common_ids: Optional[Iterable[str]] = None
) -> Tuple[pd.DataFrame, pd.DataFrame]:

    if id_col not in df_old or id_col not in df_new:
        raise KeyError(f"'{id_col}' must exist in both DataFrames")

    if common_ids is None:
        common_ids = set(df_old[id_col]) & set(df_new[id_col])
    else:
        common_ids = [rid for rid in common_ids
                      if (rid in set(df_old[id_col])) and (rid in set(df_new[id_col]))]

    if not common_ids:
        empty_sparse = pd.DataFrame(columns=[id_col])
        empty_triplet = pd.DataFrame(columns=[id_col, "column", "old_val", "new_val"])
        return empty_sparse, empty_triplet

    old_sub = df_old[df_old[id_col].isin(common_ids)].set_index(id_col)
    new_sub = df_new[df_new[id_col].isin(common_ids)].set_index(id_col)

    if columns is None:
        columns = list(set(old_sub.columns) & set(new_sub.columns))
    if id_col in columns:
        columns.remove(id_col)

    diff_mask = (old_sub[columns] != new_sub[columns])

    changed_cols = [c for c in columns if diff_mask[c].any()]
    if changed_cols:
        sparse_df = new_sub.loc[:, changed_cols].copy()
        sparse_df = sparse_df[diff_mask[changed_cols]]
        sparse_df = sparse_df.dropna(how="all")
        sparse_df = sparse_df.reset_index()
    else:
        sparse_df = pd.DataFrame(columns=[id_col])

    triplet_records: List[Dict] = []
    for col in changed_cols:
        changed_rows = diff_mask[col][diff_mask[col]].index
        for rid in changed_rows:
            triplet_records.append({
                id_col:  rid,
                "column": col,
                "old_val": old_sub.at[rid, col],
                "new_val": new_sub.at[rid, col],
            })
    triplet_df = pd.DataFrame(triplet_records,
                              columns=[id_col, "column", "old_val", "new_val"])

    return sparse_df, triplet_df


## Column Level Diffs

In [25]:
def appended_deleted_columns(df1 : pd.DataFrame, df2 : pd.DataFrame) :
    df1_columns = set(df1.columns)
    df2_columns = set(df2.columns)

    appended_columns = df2_columns - df1_columns
    deleted_columns = df1_columns - df2_columns
    unchanged_columns = df1_columns & df2_columns

    return {
        "appended" : appended_columns,
        "deleted" : deleted_columns,
        "unchanged" : unchanged_columns
    }

In [26]:
def detect_column_type_changes(df1 : pd.DataFrame, df2 : pd.DataFrame, common_columns = None):
    changes = []

    if common_columns is None:
        common_columns = set(df1.columns) & set(df2.columns)
    for col in common_columns:
        df1_dtype = df1[col].dtype
        df2_dtype = df2[col].dtype
        if df1_dtype != df2_dtype:
            changes.append({
                "column" : col,
                "old_type" : str(df1_dtype),
                "new_type" : str(df2_dtype)
            })
    return changes

## Commit Utilities

In [27]:
from pathlib import Path
import pandas as pd
import json

def write_parquet(df: pd.DataFrame, path: Path) -> bool:
    if df is not None and not df.empty:
        df.to_parquet(path)
        return True
    return False

def write_json(data, path: Path) -> bool:
    if data:
        with path.open("w") as f:
            json.dump(data, f, indent=2)
        return True
    return False


In [28]:
from typing import Dict, Optional

def build_version_metadata(
    *,
    version: int,
    parent: int,
    files: Dict[str, str],
    row_changes: Dict[str, int],
    col_changes: Dict[str, int],
    msg: str,
    timestamp: Optional[str] = None,
) -> Dict:
    return {
        "version": version,
        "parent": f"v{parent}",
        "timestamp": timestamp,
        "message": msg,
        "files": files,
        "row_changes": row_changes,
        "column_changes": col_changes,
    }


In [29]:
import json
from pathlib import Path

def append_history(root: Path, entry: dict):
    path = root / "logs" / "history.json"
    history = json.loads(path.read_text()) if path.exists() else []
    history.append(entry)
    path.write_text(json.dumps(history, indent=2))


In [30]:
def commit_version(
    root_dir: str | Path = PROJECT_FOLDER,
    *,
    version: int,
    parent: int,
    appended_rows:  Optional[pd.DataFrame] = None,
    updated_rows:   Optional[pd.DataFrame] = None,
    updated_triplet: Optional[pd.DataFrame] = None,
    deleted_rows:   Optional[pd.DataFrame] = None,
    deleted_ids:    Optional[List[str]] = None,
    col_appended:   Optional[pd.DataFrame] = None,
    col_deleted:    Optional[pd.DataFrame] = None,
    col_type_changes: Optional[List[Dict[str,str]]] = None,
    commit_message: str = "",
    timestamp: Optional[str] = None,
    snapshot_df: Optional[pd.DataFrame] = None,
) -> None:

    root  = Path(root_dir)
    vdir  = root / "versions" / f"v{version}"
    vdir.mkdir(parents=True, exist_ok= True)

    files: Dict[str, str] = {}
    row_changes = {"added": 0, "deleted": 0, "updated": 0}
    col_changes = {"added": 0, "deleted": 0, "type_changed": 0}

    if write_parquet(appended_rows, vdir / "appended.parquet"):
        files["appended"] = "appended.parquet"
        row_changes["added"] = len(appended_rows)

    if write_parquet(updated_rows, vdir / "updated.parquet"):
        files["updated"] = "updated.parquet"
        row_changes["updated"] = updated_rows["_row_id"].nunique()

    if write_parquet(updated_triplet, vdir / "updated_triplet.parquet"):
        files["updated_triplet"] = "updated_triplet.parquet"

    if write_parquet(deleted_rows, vdir / "deleted_rows.parquet"):
        files["deleted_rows"] = "deleted_rows.parquet"
        row_changes["deleted"] = len(deleted_rows)
    elif write_json(deleted_ids, vdir / "deleted_ids.json"):
        files["deleted_ids"] = "deleted_ids.json"
        row_changes["deleted"] = len(deleted_ids or [])

    if write_parquet(col_appended, vdir / "col_appended.parquet"):
        files["col_appended"] = "col_appended.parquet"
        col_changes["added"] = len(col_appended.columns) - 1

    if write_parquet(col_deleted, vdir / "col_deleted.parquet"):
        files["col_deleted"] = "col_deleted.parquet"
        col_changes["deleted"] = len(col_deleted.columns) - 1

    if write_json(col_type_changes, vdir / "col_type_changes.json"):
        files["col_type_changes"] = "col_type_changes.json"
        col_changes["type_changed"] = len(col_type_changes or [])

    meta = build_version_metadata(
        version=version, parent=parent, files=files,
        row_changes=row_changes, col_changes=col_changes,
        msg=commit_message, timestamp=timestamp)
    (vdir / "metadata.json").write_text(json.dumps(meta, indent=2))

    if snapshot_df is not None:
        snapshot_df.to_parquet(root / "snapshots" / f"v{version}.parquet", compression= "snappy")

    append_history(root, meta)
    print(f"✅ Committed v{version} (parent v{parent}) with files: {', '.join(files)}")


## Reconstruction

In [31]:
def reconstruct_version(version: int, root_dir: str | Path = PROJECT_FOLDER) -> pd.DataFrame:
    root = Path(root_dir)

    snapshot_path = root / "snapshots" / f"v{version}.parquet"
    if snapshot_path.exists():
        df = pd.read_parquet(snapshot_path)
        df["_row_id"] = df["_row_id"].astype(str)
        return df.reset_index(drop=True)

    if version == 1:
        base = pd.read_parquet(root / "base" / "v1.parquet")
        base["_row_id"] = base["_row_id"].astype(str)
        return base.reset_index(drop=True)

    meta_path = root / "versions" / f"v{version}" / "metadata.json"
    if not meta_path.exists():
        raise FileNotFoundError(f"metadata.json missing for v{version}")
    meta = json.loads(meta_path.read_text())
    parent_v = int(meta["parent"].lstrip("v"))
    df = reconstruct_version(parent_v, root_dir)

    # clean index
    df = df.reset_index(drop=True)
    if not df.index.is_unique:
        raise ValueError(f"Duplicate index while reconstructing v{version}")

    vdir = root / "versions" / f"v{version}"

    ids_p = vdir / "deleted_ids.json"
    if ids_p.exists():
        ids = json.loads(ids_p.read_text())
        df = df[~df["_row_id"].isin(ids)]

    dr_p = vdir / "deleted_rows.parquet"
    if dr_p.exists():
        ids = pd.read_parquet(dr_p)["_row_id"].astype(str)
        df = df[~df["_row_id"].isin(ids)]

    app_p = vdir / "appended.parquet"
    if app_p.exists():
        a = pd.read_parquet(app_p)
        a["_row_id"] = a["_row_id"].astype(str)
        df = pd.concat([df, a], ignore_index=True)

    up_p = vdir / "updated.parquet"
    if up_p.exists():
          up = pd.read_parquet(up_p)
          up["_row_id"] = up["_row_id"].astype(str)

          parent_cols = df.columns.tolist()

          df_idx = df.set_index("_row_id")
          up_idx = up.set_index("_row_id")
          df_idx.update(up_idx)
          df = df_idx.reset_index()
          df = df[parent_cols]


    ca_p = vdir / "col_appended.parquet"
    if ca_p.exists():
        new_cols = pd.read_parquet(ca_p)
        new_cols["_row_id"] = new_cols["_row_id"].astype(str)
        new_cols = new_cols.set_index("_row_id")
        for c in new_cols.columns:
            if c == "_row_id":
                continue
            df[c] = df["_row_id"].map(new_cols[c])

    cd_p = vdir / "col_deleted.parquet"
    if cd_p.exists():
        drop_cols = pd.read_parquet(cd_p).columns.tolist()
        drop_cols = [c for c in drop_cols if c != "_row_id"]
        df = df.drop(columns=drop_cols, errors="ignore")

    ct_p = vdir / "col_type_changes.json"
    if ct_p.exists():
        for ch in json.loads(ct_p.read_text()):
            col, new_t = ch["column"], ch["new_type"]
            if col in df.columns:
                df[col] = df[col].astype(new_t)

    return df.reset_index(drop=True)

## Dataframe Generators from row/columns

In [32]:
def get_rows_by_id( df,  row_ids, id_col = "_row_id", preserve_order = True):

    if id_col not in df.columns:
        raise KeyError(f"Column '{id_col}' not in DataFrame")

    df[id_col] = df[id_col].astype(str)
    row_ids = [str(rid) for rid in row_ids]

    id_set = set(row_ids)
    mask = df[id_col].isin(id_set)
    matched_df = df.loc[mask].copy()

    if preserve_order:
        matched_df = (
            matched_df.set_index(id_col, drop=False)
            .reindex(row_ids)
            .dropna(subset=[id_col])
            .reset_index(drop=True)
        )

    present_ids = set(matched_df[id_col])
    not_found = [rid for rid in row_ids if rid not in present_ids]

    return matched_df, not_found


In [33]:
from typing import List
import pandas as pd

def df_with_columns( df: pd.DataFrame, cols : List[str], id_col = "_row_id"):
      selected_cols = [id_col] + [col for col in cols if col in df.columns]
      missing = [col for col in selected_cols if col not in df.columns]
      return df[selected_cols].copy()

## Validation Function

In [34]:
def equal_data_only(df1, df2):
    common_cols = sorted(set(df1.columns) & set(df2.columns))
    df1_sorted = df1[common_cols].sort_values(by=common_cols).reset_index(drop=True)
    df2_sorted = df2[common_cols].sort_values(by=common_cols).reset_index(drop=True)
    return df1_sorted.equals(df2_sorted)

In [35]:
create_folder_structure(PROJECT_FOLDER)

✅ Delta Trace root ready at /content/drive/MyDrive/DeltaTrace


## Version 1

In [36]:
df_v1 = pd.read_parquet(DATASET1)
df_v1

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.60,1.0,N,229,237,1,10.00,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0,0.00
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.50,1.0,N,236,237,1,5.10,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.00
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.60,1.0,N,141,141,1,5.10,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,0.00
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,1.0,N,244,244,2,7.20,1.0,0.5,0.00,0.0,1.0,9.70,0.0,0.0,0.00
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,1.0,N,244,116,2,5.80,1.0,0.5,0.00,0.0,1.0,8.30,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3475221,2,2025-01-31 23:01:48,2025-01-31 23:16:29,NaN,3.35,NaN,None,79,237,0,15.85,0.0,0.5,0.00,0.0,1.0,20.60,NaN,NaN,0.75
3475222,2,2025-01-31 23:50:29,2025-02-01 00:17:27,NaN,8.73,NaN,None,161,116,0,28.14,0.0,0.5,0.00,0.0,1.0,32.89,NaN,NaN,0.75
3475223,2,2025-01-31 23:26:59,2025-01-31 23:43:01,NaN,2.64,NaN,None,144,246,0,14.91,0.0,0.5,0.00,0.0,1.0,19.66,NaN,NaN,0.75
3475224,2,2025-01-31 23:14:34,2025-01-31 23:34:52,NaN,3.16,NaN,None,142,107,0,17.55,0.0,0.5,0.00,0.0,1.0,22.30,NaN,NaN,0.75


In [37]:
ensure_column(df_v1)
df_v1

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,_row_id
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.60,1.0,N,229,237,1,...,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0,0.00,3ed1784f0c354620823e347e5b58c16e
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.50,1.0,N,236,237,1,...,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.00,717898bdbdca4b19b7e31e43551c9f0d
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.60,1.0,N,141,141,1,...,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,0.00,96de66ede0ef44afb962111474f233a8
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,1.0,N,244,244,2,...,1.0,0.5,0.00,0.0,1.0,9.70,0.0,0.0,0.00,f8a335c3110842a28d4d8df4a4cc202a
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,1.0,N,244,116,2,...,1.0,0.5,0.00,0.0,1.0,8.30,0.0,0.0,0.00,8b343719c7b64611a27260ecf3977cb2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3475221,2,2025-01-31 23:01:48,2025-01-31 23:16:29,NaN,3.35,NaN,None,79,237,0,...,0.0,0.5,0.00,0.0,1.0,20.60,NaN,NaN,0.75,1ff8e38e194f4ccbbb448eaacc0618b9
3475222,2,2025-01-31 23:50:29,2025-02-01 00:17:27,NaN,8.73,NaN,None,161,116,0,...,0.0,0.5,0.00,0.0,1.0,32.89,NaN,NaN,0.75,b0f90718b366410d8cea7942ba2ffcab
3475223,2,2025-01-31 23:26:59,2025-01-31 23:43:01,NaN,2.64,NaN,None,144,246,0,...,0.0,0.5,0.00,0.0,1.0,19.66,NaN,NaN,0.75,55748e25c44847a98e26a42a8e2bad36
3475224,2,2025-01-31 23:14:34,2025-01-31 23:34:52,NaN,3.16,NaN,None,142,107,0,...,0.0,0.5,0.00,0.0,1.0,22.30,NaN,NaN,0.75,e99a1b48174a4a50b1a9da6c127e13c7


In [38]:
df_v1.to_parquet(PROJECT_FOLDER / "base" / "v1.parquet", compression="snappy", index=False)

## Version 2

In [39]:
df2 = pd.read_parquet(DATASET2)
df2

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,2,2025-02-01 00:12:18,2025-02-01 00:32:33,3.0,3.12,1.0,N,246,79,1,19.80,1.00,0.5,5.11,0.0,1.0,30.66,2.5,0.0,0.75
1,2,2025-02-01 00:40:04,2025-02-01 00:49:15,1.0,1.40,1.0,N,114,79,1,10.00,1.00,0.5,3.15,0.0,1.0,18.90,2.5,0.0,0.75
2,1,2025-02-01 00:06:09,2025-02-01 00:11:51,0.0,0.40,1.0,N,211,144,1,6.50,4.25,0.5,1.00,0.0,1.0,13.25,2.5,0.0,0.75
3,1,2025-02-01 00:15:13,2025-02-01 00:20:19,0.0,0.70,1.0,N,113,249,1,7.20,4.25,0.5,2.00,0.0,1.0,14.95,2.5,0.0,0.75
4,2,2025-02-01 00:02:52,2025-02-01 00:20:25,1.0,4.19,1.0,N,113,263,1,19.80,1.00,0.5,5.11,0.0,1.0,30.66,2.5,0.0,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577538,2,2025-02-28 23:39:09,2025-03-01 00:05:17,NaN,13.73,NaN,None,209,127,0,47.56,0.00,0.5,0.00,0.0,1.0,52.31,NaN,NaN,0.75
3577539,2,2025-02-28 23:45:00,2025-02-28 23:56:22,NaN,1.39,NaN,None,4,234,0,-4.75,0.00,0.5,0.00,0.0,1.0,2.36,NaN,NaN,0.75
3577540,2,2025-02-28 23:28:19,2025-02-28 23:38:18,NaN,1.14,NaN,None,107,4,0,12.68,0.00,0.5,0.00,0.0,1.0,17.43,NaN,NaN,0.75
3577541,2,2025-02-28 23:46:52,2025-03-01 00:02:48,NaN,7.43,NaN,None,50,243,0,-4.75,0.00,0.5,0.00,0.0,1.0,5.17,NaN,NaN,0.75


In [40]:
df_v2 = pd.concat([df_v1, df2[:1000]])
ensure_column(df_v2)
df_v2

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,_row_id
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.60,1.0,N,229,237,1,...,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0,0.00,3ed1784f0c354620823e347e5b58c16e
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.50,1.0,N,236,237,1,...,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.00,717898bdbdca4b19b7e31e43551c9f0d
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.60,1.0,N,141,141,1,...,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,0.00,96de66ede0ef44afb962111474f233a8
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,1.0,N,244,244,2,...,1.0,0.5,0.00,0.0,1.0,9.70,0.0,0.0,0.00,f8a335c3110842a28d4d8df4a4cc202a
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,1.0,N,244,116,2,...,1.0,0.5,0.00,0.0,1.0,8.30,0.0,0.0,0.00,8b343719c7b64611a27260ecf3977cb2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,2025-02-01 00:26:45,2025-02-01 00:32:21,1.0,0.77,1.0,N,151,24,1,...,1.0,0.5,1.94,0.0,1.0,11.64,0.0,0.0,0.00,58de771f69404f5c8a92e6a9452b9dc3
996,2,2025-02-01 00:09:21,2025-02-01 00:25:31,1.0,1.82,1.0,N,79,231,1,...,1.0,0.5,4.27,0.0,1.0,25.62,2.5,0.0,0.75,d8272b95f0184cefa4109910f0737013
997,2,2025-02-01 00:26:14,2025-02-01 00:35:32,1.0,1.98,1.0,N,231,87,1,...,1.0,0.5,2.20,0.0,1.0,20.05,2.5,0.0,0.75,89d77fed31204804b4e2499c663b4ff0
998,2,2025-02-01 00:48:47,2025-02-01 00:49:51,1.0,0.16,1.0,N,87,209,4,...,-1.0,-0.5,0.00,0.0,-1.0,-9.45,-2.5,0.0,-0.75,95ab638846b8412fb44c8fb9c0115b61


In [41]:
h = appended_deleted_rows(df_v1, df_v2)
print("Changes between V1 and V2")
print(len(h["appended"]), len(h["deleted"]), len(h["unchanged"]))

Changes between V1 and V2
1000 0 3475226


In [42]:
appended_df = get_rows_by_id(df_v2, list(h["appended"]))[0]
appended_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,_row_id
0,2,2025-02-01 00:29:04,2025-02-01 00:40:17,1.0,3.69,1.0,N,239,116,1,...,1.0,0.5,6.81,0.0,1.0,29.51,2.5,0.0,0.00,09eb0a9643a548ffb72757b6c564b1c5
1,2,2025-02-01 00:07:58,2025-02-01 00:21:29,4.0,1.52,1.0,N,107,148,1,...,1.0,0.5,0.00,0.0,1.0,19.25,2.5,0.0,0.75,5f345a66e5554f5c8f608e7a206f6180
2,2,2025-02-01 00:40:07,2025-02-01 00:42:53,1.0,0.48,1.0,N,48,50,1,...,1.0,0.5,1.00,0.0,1.0,11.85,2.5,0.0,0.75,ea791a4ecd2b4857bc9bf93a522ac8af
3,2,2025-02-01 00:12:13,2025-02-01 00:18:32,1.0,0.73,1.0,N,249,211,1,...,1.0,0.5,1.00,0.0,1.0,13.95,2.5,0.0,0.75,de8e521f3cb64c77a28ca2ce41680cc7
4,2,2025-02-01 00:47:15,2025-02-01 00:49:43,1.0,0.59,1.0,N,90,100,1,...,1.0,0.5,2.17,0.0,1.0,13.02,2.5,0.0,0.75,4faeb5fe7cf54b8abdd8e334ad7fc346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,2025-02-01 00:50:42,2025-02-01 01:07:19,1.0,2.74,1.0,N,158,233,1,...,1.0,0.5,4.69,0.0,1.0,28.14,2.5,0.0,0.75,5a68690630044361b4d77a551367def1
996,2,2025-02-01 00:25:35,2025-02-01 00:34:18,1.0,1.70,1.0,N,163,236,1,...,1.0,0.5,3.43,0.0,1.0,20.58,2.5,0.0,0.75,6baaccfd5c0045328e829019b166fbe6
997,2,2025-02-01 00:15:12,2025-02-01 00:30:44,1.0,3.54,1.0,N,249,229,1,...,1.0,0.5,4.83,0.0,1.0,28.98,2.5,0.0,0.75,c0dc18ccad7f4d728245116d15c20d84
998,2,2025-02-01 00:07:05,2025-02-01 00:21:34,1.0,2.47,1.0,N,148,66,1,...,1.0,0.5,4.27,0.0,1.0,25.62,2.5,0.0,0.75,1e18504602184edda06e5c3ffdaac7a0


In [43]:
commit_version(PROJECT_FOLDER, version = 2,parent = 1, appended_rows= appended_df)

✅ Committed v2 (parent v1) with files: appended


In [44]:
# v2 = reconstruct_version(2)
# v2

In [45]:
# equal_data_only(v2, df_v2)

## Version  3

In [46]:
df_v3 = df_v2[df_v2['VendorID'] != 1]
ensure_column(df_v3)
df_v3

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,_row_id
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,1.0,N,244,244,2,...,1.0,0.5,0.00,0.0,1.0,9.70,0.0,0.0,0.00,f8a335c3110842a28d4d8df4a4cc202a
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,1.0,N,244,116,2,...,1.0,0.5,0.00,0.0,1.0,8.30,0.0,0.0,0.00,8b343719c7b64611a27260ecf3977cb2
5,2,2025-01-01 00:48:24,2025-01-01 01:08:26,2.0,2.63,1.0,N,239,68,2,...,1.0,0.5,0.00,0.0,1.0,24.10,2.5,0.0,0.00,af9926b56fba472c9c0da5cac7b774ec
9,2,2025-01-01 00:00:02,2025-01-01 00:09:36,1.0,1.71,1.0,N,237,262,2,...,1.0,0.5,0.00,0.0,1.0,16.40,2.5,0.0,0.00,a4353a21fb474e98bb7543e30a357b5b
10,2,2025-01-01 00:20:28,2025-01-01 00:28:04,1.0,2.29,1.0,N,237,75,2,...,1.0,0.5,0.00,0.0,1.0,16.40,2.5,0.0,0.00,b2eedaf0e74643ca86ad73dec40b0cbb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,2025-02-01 00:26:45,2025-02-01 00:32:21,1.0,0.77,1.0,N,151,24,1,...,1.0,0.5,1.94,0.0,1.0,11.64,0.0,0.0,0.00,58de771f69404f5c8a92e6a9452b9dc3
996,2,2025-02-01 00:09:21,2025-02-01 00:25:31,1.0,1.82,1.0,N,79,231,1,...,1.0,0.5,4.27,0.0,1.0,25.62,2.5,0.0,0.75,d8272b95f0184cefa4109910f0737013
997,2,2025-02-01 00:26:14,2025-02-01 00:35:32,1.0,1.98,1.0,N,231,87,1,...,1.0,0.5,2.20,0.0,1.0,20.05,2.5,0.0,0.75,89d77fed31204804b4e2499c663b4ff0
998,2,2025-02-01 00:48:47,2025-02-01 00:49:51,1.0,0.16,1.0,N,87,209,4,...,-1.0,-0.5,0.00,0.0,-1.0,-9.45,-2.5,0.0,-0.75,95ab638846b8412fb44c8fb9c0115b61


In [47]:
h = appended_deleted_rows(df_v1, df_v3)
print("Changes between V1 and V3")
print(len(h["appended"]), len(h["deleted"]), len(h["unchanged"]))

h = appended_deleted_rows(df_v2, df_v3)
print("Changes between V2 and V3")
print(len(h["appended"]), len(h["deleted"]), len(h["unchanged"]))

Changes between V1 and V3
735 753671 2721555
Changes between V2 and V3
0 753936 2722290


In [48]:
deleted_df = get_rows_by_id(df_v2, list(h["deleted"]))[0]
deleted_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,_row_id
0,1,2025-01-23 08:21:48,2025-01-23 08:29:47,1.0,1.0,1.0,N,48,163,1,...,3.25,0.5,5.00,0.00,1.0,17.65,2.5,0.00,0.75,c29d19763e4143ec90ed6690c0591a37
1,1,2025-01-15 13:01:12,2025-01-15 13:21:50,1.0,1.7,1.0,N,163,234,1,...,3.25,0.5,4.63,0.00,1.0,27.78,2.5,0.00,0.75,59b5d68e6bab49d58af12b29eda970cb
2,1,2025-01-11 15:41:42,2025-01-11 15:53:38,1.0,1.0,1.0,N,234,186,1,...,3.25,0.5,0.00,0.00,1.0,16.15,2.5,0.00,0.75,d96a8c2c89ba42a487a49f8dc8dafd70
3,1,2025-01-02 18:29:44,2025-01-02 18:50:59,1.0,5.0,1.0,N,138,202,1,...,9.25,0.5,5.00,0.00,1.0,40.45,0.0,1.75,0.00,8049e26b31d24db88fd71af016438132
4,1,2025-01-18 18:06:19,2025-01-18 18:22:02,1.0,2.6,1.0,N,229,50,1,...,3.25,0.5,3.80,0.00,1.0,22.75,2.5,0.00,0.75,5b8509db0aab454d800280ec1833fa76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753931,1,2025-01-17 07:26:29,2025-01-17 07:28:51,1.0,0.4,1.0,N,163,50,1,...,3.25,0.5,0.00,0.00,1.0,9.15,2.5,0.00,0.75,d0817e566eb846318ac320c578f90636
753932,1,2025-01-14 20:36:25,2025-01-14 20:54:26,1.0,2.4,1.0,N,230,263,1,...,4.25,0.5,4.15,0.00,1.0,24.80,2.5,0.00,0.75,8098a500228848c78bf90b0d3d96b5db
753933,1,2025-01-20 20:32:25,2025-01-20 20:40:50,1.0,1.1,1.0,N,162,237,1,...,4.25,0.5,2.00,0.00,1.0,16.35,2.5,0.00,0.75,750072da0d7d4997b4ed294e39b7b1e4
753934,1,2025-01-03 15:44:13,2025-01-03 15:52:25,1.0,0.8,1.0,N,88,13,1,...,2.50,0.5,2.40,0.00,1.0,14.30,2.5,0.00,0.00,c6a73b6804014f64b7ddee9dd9fdaa1d


In [49]:
commit_version(PROJECT_FOLDER, version = 3,parent = 2, deleted_rows= deleted_df, deleted_ids= list(h["deleted"]))

✅ Committed v3 (parent v2) with files: deleted_rows


In [50]:
# v3 = reconstruct_version(3)
# v3

In [51]:
# equal_data_only(v3, df_v3)

## Version 4

In [52]:
import random

RANDOM_SEED = 42
random.seed(RANDOM_SEED)

ids = df_v3["_row_id"].to_list()
num_updates = 10023
updating_ids = random.sample(ids, num_updates)

df_v4 = df_v3.copy()

first = updating_ids[:num_updates//2]
second = updating_ids[num_updates//2:]

df_v4.loc[df_v4["_row_id"].isin(first), "tip_amount"] += random.randrange(1, 2)
df_v4.loc[df_v4["_row_id"].isin(second), "improvement_surcharge"] = 2
df_v4

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,_row_id
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,1.0,N,244,244,2,...,1.0,0.5,0.00,0.0,1.0,9.70,0.0,0.0,0.00,f8a335c3110842a28d4d8df4a4cc202a
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,1.0,N,244,116,2,...,1.0,0.5,0.00,0.0,1.0,8.30,0.0,0.0,0.00,8b343719c7b64611a27260ecf3977cb2
5,2,2025-01-01 00:48:24,2025-01-01 01:08:26,2.0,2.63,1.0,N,239,68,2,...,1.0,0.5,0.00,0.0,1.0,24.10,2.5,0.0,0.00,af9926b56fba472c9c0da5cac7b774ec
9,2,2025-01-01 00:00:02,2025-01-01 00:09:36,1.0,1.71,1.0,N,237,262,2,...,1.0,0.5,0.00,0.0,1.0,16.40,2.5,0.0,0.00,a4353a21fb474e98bb7543e30a357b5b
10,2,2025-01-01 00:20:28,2025-01-01 00:28:04,1.0,2.29,1.0,N,237,75,2,...,1.0,0.5,0.00,0.0,1.0,16.40,2.5,0.0,0.00,b2eedaf0e74643ca86ad73dec40b0cbb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,2025-02-01 00:26:45,2025-02-01 00:32:21,1.0,0.77,1.0,N,151,24,1,...,1.0,0.5,1.94,0.0,1.0,11.64,0.0,0.0,0.00,58de771f69404f5c8a92e6a9452b9dc3
996,2,2025-02-01 00:09:21,2025-02-01 00:25:31,1.0,1.82,1.0,N,79,231,1,...,1.0,0.5,4.27,0.0,1.0,25.62,2.5,0.0,0.75,d8272b95f0184cefa4109910f0737013
997,2,2025-02-01 00:26:14,2025-02-01 00:35:32,1.0,1.98,1.0,N,231,87,1,...,1.0,0.5,2.20,0.0,1.0,20.05,2.5,0.0,0.75,89d77fed31204804b4e2499c663b4ff0
998,2,2025-02-01 00:48:47,2025-02-01 00:49:51,1.0,0.16,1.0,N,87,209,4,...,-1.0,-0.5,0.00,0.0,-1.0,-9.45,-2.5,0.0,-0.75,95ab638846b8412fb44c8fb9c0115b61


In [53]:
h = appended_deleted_rows(df_v1, df_v4)
print("Changes between V1 and V4")
print(len(h["appended"]), len(h["deleted"]), len(h["unchanged"]))

h = appended_deleted_rows(df_v3, df_v4)
print("Changes between V3 and V4")
print(len(h["appended"]), len(h["deleted"]), len(h["unchanged"]))

Changes between V1 and V4
735 753671 2721555
Changes between V3 and V4
0 0 2722290


In [54]:
sparse, triplet = detect_updates(df_v3, df_v4)

In [55]:
sparse

,_row_id,improvement_surcharge,congestion_surcharge,tip_amount,RatecodeID,store_and_fwd_flag,Airport_fee,passenger_count
0,0d03a811bab547cdb9e1d339d9cff32e,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,6195535f77c74248a4618eeaf2c0cb5f,NaN,NaN,1.0,NaN,NaN,NaN,NaN
2,d3fc789091da479e8eb1941ddeee59bf,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,97f796b5f6234a14847f0ccc933ec168,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,c15cbc3d1e6c42638941f7b77f66e539,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
10018,b9c1b1c765a4413ea63f369b902c4cb6,2.0,NaN,NaN,NaN,None,NaN,NaN
10019,ed786bf26739489da834b4cb0f555513,2.0,NaN,NaN,NaN,None,NaN,NaN
10020,4ebc8e60e3a543d080378429bf3525bc,NaN,NaN,1.0,NaN,None,NaN,NaN
10021,b014124ed1a14491a5025d29d9d2a52c,NaN,NaN,3.6,NaN,NaN,NaN,NaN


In [56]:
triplet

,_row_id,column,old_val,new_val
0,0d03a811bab547cdb9e1d339d9cff32e,improvement_surcharge,1.0,2.0
1,d3fc789091da479e8eb1941ddeee59bf,improvement_surcharge,1.0,2.0
2,97f796b5f6234a14847f0ccc933ec168,improvement_surcharge,1.0,2.0
3,c15cbc3d1e6c42638941f7b77f66e539,improvement_surcharge,1.0,2.0
4,79281db149ea430d8bb20d442aa05c68,improvement_surcharge,1.0,2.0
...,...,...,...,...
2269743,1ff8e38e194f4ccbbb448eaacc0618b9,passenger_count,NaN,NaN
2269744,b0f90718b366410d8cea7942ba2ffcab,passenger_count,NaN,NaN
2269745,55748e25c44847a98e26a42a8e2bad36,passenger_count,NaN,NaN
2269746,e99a1b48174a4a50b1a9da6c127e13c7,passenger_count,NaN,NaN


In [57]:
commit_version(PROJECT_FOLDER, version = 4, parent = 3, updated_rows= sparse, updated_triplet= triplet)

✅ Committed v4 (parent v3) with files: updated, updated_triplet


In [58]:
# v4 = reconstruct_version(4)
# v4

In [59]:
# equal_data_only(v4 , df_v4)

## Version 5

In [60]:
df_v5 = df_v4.copy()
ensure_column(df_v5)
df_v5["trip_duration"] = df_v5["tpep_dropoff_datetime"] - df_v5["tpep_pickup_datetime"]
df_v5

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,_row_id,trip_duration
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,1.0,N,244,244,2,...,0.5,0.00,0.0,1.0,9.70,0.0,0.0,0.00,f8a335c3110842a28d4d8df4a4cc202a,0 days 00:05:34
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,1.0,N,244,116,2,...,0.5,0.00,0.0,1.0,8.30,0.0,0.0,0.00,8b343719c7b64611a27260ecf3977cb2,0 days 00:03:32
5,2,2025-01-01 00:48:24,2025-01-01 01:08:26,2.0,2.63,1.0,N,239,68,2,...,0.5,0.00,0.0,1.0,24.10,2.5,0.0,0.00,af9926b56fba472c9c0da5cac7b774ec,0 days 00:20:02
9,2,2025-01-01 00:00:02,2025-01-01 00:09:36,1.0,1.71,1.0,N,237,262,2,...,0.5,0.00,0.0,1.0,16.40,2.5,0.0,0.00,a4353a21fb474e98bb7543e30a357b5b,0 days 00:09:34
10,2,2025-01-01 00:20:28,2025-01-01 00:28:04,1.0,2.29,1.0,N,237,75,2,...,0.5,0.00,0.0,1.0,16.40,2.5,0.0,0.00,b2eedaf0e74643ca86ad73dec40b0cbb,0 days 00:07:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,2025-02-01 00:26:45,2025-02-01 00:32:21,1.0,0.77,1.0,N,151,24,1,...,0.5,1.94,0.0,1.0,11.64,0.0,0.0,0.00,58de771f69404f5c8a92e6a9452b9dc3,0 days 00:05:36
996,2,2025-02-01 00:09:21,2025-02-01 00:25:31,1.0,1.82,1.0,N,79,231,1,...,0.5,4.27,0.0,1.0,25.62,2.5,0.0,0.75,d8272b95f0184cefa4109910f0737013,0 days 00:16:10
997,2,2025-02-01 00:26:14,2025-02-01 00:35:32,1.0,1.98,1.0,N,231,87,1,...,0.5,2.20,0.0,1.0,20.05,2.5,0.0,0.75,89d77fed31204804b4e2499c663b4ff0,0 days 00:09:18
998,2,2025-02-01 00:48:47,2025-02-01 00:49:51,1.0,0.16,1.0,N,87,209,4,...,-0.5,0.00,0.0,-1.0,-9.45,-2.5,0.0,-0.75,95ab638846b8412fb44c8fb9c0115b61,0 days 00:01:04


In [61]:
col_changes = appended_deleted_columns(df_v4, df_v5)
col_changes

{'appended': {'trip_duration'},
 'deleted': set(),
 'unchanged': {'Airport_fee',
  'DOLocationID',
  'PULocationID',
  'RatecodeID',
  'VendorID',
  '_row_id',
  'cbd_congestion_fee',
  'congestion_surcharge',
  'extra',
  'fare_amount',
  'improvement_surcharge',
  'mta_tax',
  'passenger_count',
  'payment_type',
  'store_and_fwd_flag',
  'tip_amount',
  'tolls_amount',
  'total_amount',
  'tpep_dropoff_datetime',
  'tpep_pickup_datetime',
  'trip_distance'}}

In [62]:
appended_col_df = df_with_columns(df_v5, col_changes["appended"])
appended_col_df

,_row_id,trip_duration
3,f8a335c3110842a28d4d8df4a4cc202a,0 days 00:05:34
4,8b343719c7b64611a27260ecf3977cb2,0 days 00:03:32
5,af9926b56fba472c9c0da5cac7b774ec,0 days 00:20:02
9,a4353a21fb474e98bb7543e30a357b5b,0 days 00:09:34
10,b2eedaf0e74643ca86ad73dec40b0cbb,0 days 00:07:36
...,...,...
995,58de771f69404f5c8a92e6a9452b9dc3,0 days 00:05:36
996,d8272b95f0184cefa4109910f0737013,0 days 00:16:10
997,89d77fed31204804b4e2499c663b4ff0,0 days 00:09:18
998,95ab638846b8412fb44c8fb9c0115b61,0 days 00:01:04


In [63]:
commit_version(version = 5, parent = 4, col_appended = appended_col_df)

✅ Committed v5 (parent v4) with files: col_appended


In [64]:
# v5 = reconstruct_version(5)
# v5

In [65]:
# equal_data_only(v5, df_v5)

## Version 6

In [66]:
df_v6 = df_v5.copy()
df_v6 = df_v6.drop(columns = "RatecodeID")
df_v6

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,_row_id,trip_duration
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,N,244,244,2,7.2,...,0.5,0.00,0.0,1.0,9.70,0.0,0.0,0.00,f8a335c3110842a28d4d8df4a4cc202a,0 days 00:05:34
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,N,244,116,2,5.8,...,0.5,0.00,0.0,1.0,8.30,0.0,0.0,0.00,8b343719c7b64611a27260ecf3977cb2,0 days 00:03:32
5,2,2025-01-01 00:48:24,2025-01-01 01:08:26,2.0,2.63,N,239,68,2,19.1,...,0.5,0.00,0.0,1.0,24.10,2.5,0.0,0.00,af9926b56fba472c9c0da5cac7b774ec,0 days 00:20:02
9,2,2025-01-01 00:00:02,2025-01-01 00:09:36,1.0,1.71,N,237,262,2,11.4,...,0.5,0.00,0.0,1.0,16.40,2.5,0.0,0.00,a4353a21fb474e98bb7543e30a357b5b,0 days 00:09:34
10,2,2025-01-01 00:20:28,2025-01-01 00:28:04,1.0,2.29,N,237,75,2,11.4,...,0.5,0.00,0.0,1.0,16.40,2.5,0.0,0.00,b2eedaf0e74643ca86ad73dec40b0cbb,0 days 00:07:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,2025-02-01 00:26:45,2025-02-01 00:32:21,1.0,0.77,N,151,24,1,7.2,...,0.5,1.94,0.0,1.0,11.64,0.0,0.0,0.00,58de771f69404f5c8a92e6a9452b9dc3,0 days 00:05:36
996,2,2025-02-01 00:09:21,2025-02-01 00:25:31,1.0,1.82,N,79,231,1,15.6,...,0.5,4.27,0.0,1.0,25.62,2.5,0.0,0.75,d8272b95f0184cefa4109910f0737013,0 days 00:16:10
997,2,2025-02-01 00:26:14,2025-02-01 00:35:32,1.0,1.98,N,231,87,1,12.1,...,0.5,2.20,0.0,1.0,20.05,2.5,0.0,0.75,89d77fed31204804b4e2499c663b4ff0,0 days 00:09:18
998,2,2025-02-01 00:48:47,2025-02-01 00:49:51,1.0,0.16,N,87,209,4,-3.7,...,-0.5,0.00,0.0,-1.0,-9.45,-2.5,0.0,-0.75,95ab638846b8412fb44c8fb9c0115b61,0 days 00:01:04


In [67]:
col_changes = appended_deleted_columns(df_v5, df_v6)
col_changes

{'appended': set(),
 'deleted': {'RatecodeID'},
 'unchanged': {'Airport_fee',
  'DOLocationID',
  'PULocationID',
  'VendorID',
  '_row_id',
  'cbd_congestion_fee',
  'congestion_surcharge',
  'extra',
  'fare_amount',
  'improvement_surcharge',
  'mta_tax',
  'passenger_count',
  'payment_type',
  'store_and_fwd_flag',
  'tip_amount',
  'tolls_amount',
  'total_amount',
  'tpep_dropoff_datetime',
  'tpep_pickup_datetime',
  'trip_distance',
  'trip_duration'}}

In [68]:
deleted_col_df = df_with_columns(df_v5, col_changes["deleted"])
deleted_col_df

,_row_id,RatecodeID
3,f8a335c3110842a28d4d8df4a4cc202a,1.0
4,8b343719c7b64611a27260ecf3977cb2,1.0
5,af9926b56fba472c9c0da5cac7b774ec,1.0
9,a4353a21fb474e98bb7543e30a357b5b,1.0
10,b2eedaf0e74643ca86ad73dec40b0cbb,1.0
...,...,...
995,58de771f69404f5c8a92e6a9452b9dc3,1.0
996,d8272b95f0184cefa4109910f0737013,1.0
997,89d77fed31204804b4e2499c663b4ff0,1.0
998,95ab638846b8412fb44c8fb9c0115b61,1.0


In [69]:
commit_version(version = 6, parent = 5, col_deleted= deleted_col_df)

✅ Committed v6 (parent v5) with files: col_deleted


In [70]:
# v6 = reconstruct_version(6)
# v6

In [71]:
# equal_data_only(v6, df_v6)

## Version 7

In [72]:
df_v6['PULocationID'].dtype

dtype('int32')

In [73]:
df_v7 = df_v6.copy()
df_v7['PULocationID'] = df_v6['PULocationID'].astype("int8")
ensure_column(df_v7)
df_v7

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,_row_id,trip_duration
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,N,-12,244,2,7.2,...,0.5,0.00,0.0,1.0,9.70,0.0,0.0,0.00,f8a335c3110842a28d4d8df4a4cc202a,0 days 00:05:34
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,N,-12,116,2,5.8,...,0.5,0.00,0.0,1.0,8.30,0.0,0.0,0.00,8b343719c7b64611a27260ecf3977cb2,0 days 00:03:32
5,2,2025-01-01 00:48:24,2025-01-01 01:08:26,2.0,2.63,N,-17,68,2,19.1,...,0.5,0.00,0.0,1.0,24.10,2.5,0.0,0.00,af9926b56fba472c9c0da5cac7b774ec,0 days 00:20:02
9,2,2025-01-01 00:00:02,2025-01-01 00:09:36,1.0,1.71,N,-19,262,2,11.4,...,0.5,0.00,0.0,1.0,16.40,2.5,0.0,0.00,a4353a21fb474e98bb7543e30a357b5b,0 days 00:09:34
10,2,2025-01-01 00:20:28,2025-01-01 00:28:04,1.0,2.29,N,-19,75,2,11.4,...,0.5,0.00,0.0,1.0,16.40,2.5,0.0,0.00,b2eedaf0e74643ca86ad73dec40b0cbb,0 days 00:07:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,2025-02-01 00:26:45,2025-02-01 00:32:21,1.0,0.77,N,-105,24,1,7.2,...,0.5,1.94,0.0,1.0,11.64,0.0,0.0,0.00,58de771f69404f5c8a92e6a9452b9dc3,0 days 00:05:36
996,2,2025-02-01 00:09:21,2025-02-01 00:25:31,1.0,1.82,N,79,231,1,15.6,...,0.5,4.27,0.0,1.0,25.62,2.5,0.0,0.75,d8272b95f0184cefa4109910f0737013,0 days 00:16:10
997,2,2025-02-01 00:26:14,2025-02-01 00:35:32,1.0,1.98,N,-25,87,1,12.1,...,0.5,2.20,0.0,1.0,20.05,2.5,0.0,0.75,89d77fed31204804b4e2499c663b4ff0,0 days 00:09:18
998,2,2025-02-01 00:48:47,2025-02-01 00:49:51,1.0,0.16,N,87,209,4,-3.7,...,-0.5,0.00,0.0,-1.0,-9.45,-2.5,0.0,-0.75,95ab638846b8412fb44c8fb9c0115b61,0 days 00:01:04


In [74]:
col_changes = detect_column_type_changes(df_v6, df_v7)
col_changes

[{'column': 'PULocationID', 'old_type': 'int32', 'new_type': 'int8'}]

In [75]:
commit_version(version = 7, parent = 6, col_type_changes= col_changes)

✅ Committed v7 (parent v6) with files: col_type_changes


In [76]:
# v7 = reconstruct_version(7)
# v7

In [77]:
# equal_data_only(v7, df_v7)

## Version 8

In [78]:
v5_length = len(df_v5)
df_v8 = df_v5[v5_length//2: int(v5_length * 1.5)]
df_v8

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,_row_id,trip_duration
1753095,2,2025-01-19 21:27:41,2025-01-19 21:37:42,1.0,1.18,1.0,N,230,161,1,...,0.5,3.15,0.0,1.0,18.90,2.5,0.0,0.75,ad58fff8e8054647bb0ac6290c4c9cf0,0 days 00:10:01
1753096,2,2025-01-19 21:42:22,2025-01-19 21:52:29,1.0,2.44,1.0,N,162,238,1,...,0.5,1.00,0.0,1.0,20.25,2.5,0.0,0.75,2176de14428b4ce8b3a529b339285d93,0 days 00:10:07
1753097,2,2025-01-19 21:07:47,2025-01-19 21:17:30,1.0,1.95,1.0,N,140,170,1,...,0.5,3.57,0.0,1.0,21.42,2.5,0.0,0.75,846c0be1c55543c793fdbe78dea586d4,0 days 00:09:43
1753098,2,2025-01-19 21:30:28,2025-01-19 21:33:48,4.0,0.65,1.0,N,141,262,1,...,0.5,1.08,0.0,1.0,11.88,2.5,0.0,0.00,de9e034fe3854830ba383b1dbb7aa9b3,0 days 00:03:20
1753099,2,2025-01-19 21:47:05,2025-01-19 21:56:20,2.0,2.68,1.0,N,141,107,1,...,0.5,4.64,0.0,1.0,23.19,2.5,0.0,0.75,b2a8f6ad08b24b0b83cd879656ce0987,0 days 00:09:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,2025-02-01 00:26:45,2025-02-01 00:32:21,1.0,0.77,1.0,N,151,24,1,...,0.5,1.94,0.0,1.0,11.64,0.0,0.0,0.00,58de771f69404f5c8a92e6a9452b9dc3,0 days 00:05:36
996,2,2025-02-01 00:09:21,2025-02-01 00:25:31,1.0,1.82,1.0,N,79,231,1,...,0.5,4.27,0.0,1.0,25.62,2.5,0.0,0.75,d8272b95f0184cefa4109910f0737013,0 days 00:16:10
997,2,2025-02-01 00:26:14,2025-02-01 00:35:32,1.0,1.98,1.0,N,231,87,1,...,0.5,2.20,0.0,1.0,20.05,2.5,0.0,0.75,89d77fed31204804b4e2499c663b4ff0,0 days 00:09:18
998,2,2025-02-01 00:48:47,2025-02-01 00:49:51,1.0,0.16,1.0,N,87,209,4,...,-0.5,0.00,0.0,-1.0,-9.45,-2.5,0.0,-0.75,95ab638846b8412fb44c8fb9c0115b61,0 days 00:01:04


In [79]:
h = appended_deleted_rows(df_v5, df_v8)
print("Changes between V5 and V8")
print(len(h["appended"]), len(h["deleted"]), len(h["unchanged"]))

Changes between V5 and V8
0 1361145 1361145


In [80]:
deleted_rows = get_rows_by_id(df_v5, list(h["deleted"]))[0]
deleted_rows

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,_row_id,trip_duration
0,2,2025-01-04 03:02:58,2025-01-04 03:06:11,1.0,0.88,1.0,N,249,186,1,...,0.5,2.30,0.0,1.0,13.80,2.5,0.00,0.00,b5dcbbc3f7d14615aab436758721afc9,0 days 00:03:13
1,2,2025-01-02 18:18:19,2025-01-02 18:28:01,1.0,1.97,1.0,N,43,162,1,...,0.5,3.72,0.0,1.0,22.32,2.5,0.00,0.00,1f1e97e46a8e434da50f719067d88f27,0 days 00:09:42
2,2,2025-01-13 08:10:47,2025-01-13 08:21:32,1.0,1.94,1.0,N,166,239,1,...,0.5,2.52,0.0,1.0,19.32,2.5,0.00,0.00,8822f3007cda457bbe2328b06858dafa,0 days 00:10:45
3,2,2025-01-16 09:36:14,2025-01-16 09:56:21,2.0,1.74,1.0,N,170,107,4,...,0.5,0.00,0.0,1.0,22.45,2.5,0.00,0.75,50e34b5972b84bb98b7e75dd0467ca9a,0 days 00:20:07
4,2,2025-01-08 21:48:38,2025-01-08 21:54:06,1.0,1.02,1.0,N,162,141,1,...,0.5,1.00,0.0,1.0,14.65,2.5,0.00,0.75,a0bfcf62c8454ae9a73dba5db86bea8f,0 days 00:05:28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361140,2,2025-01-14 06:27:25,2025-01-14 06:36:10,1.0,1.90,1.0,N,162,263,1,...,0.5,2.00,0.0,1.0,18.15,2.5,0.00,0.75,86cc15208a6741839395951c7a4da8fd,0 days 00:08:45
1361141,2,2025-01-18 16:20:57,2025-01-18 17:19:46,1.0,21.79,1.0,N,132,33,1,...,0.5,17.42,0.0,1.0,106.27,0.0,1.75,0.00,fb76a3b6104a4bc9ab70b203b3534811,0 days 00:58:49
1361142,2,2025-01-02 13:56:51,2025-01-02 14:20:30,1.0,11.08,1.0,N,219,165,1,...,0.5,5.00,0.0,1.0,51.50,0.0,0.00,0.00,fc6176663df64d28bac2d0f77a033aa1,0 days 00:23:39
1361143,2,2025-01-07 15:22:17,2025-01-07 15:38:52,1.0,1.14,1.0,N,186,230,1,...,0.5,5.00,0.0,1.0,24.65,2.5,0.00,0.75,e88ac8a0d47a4b948c6f88616cb9f33f,0 days 00:16:35


In [81]:
commit_version(version = 8, parent = 5, deleted_rows= deleted_rows)

✅ Committed v8 (parent v5) with files: deleted_rows


### Validate all versions

In [82]:
for v in range(1, 9):
    reconstructed = reconstruct_version(v)
    original = globals().get(f"df_v{v}")
    if original is not None:
        assert equal_data_only(reconstructed, original), f"Version {v} reconstruction failed"
    print(f"✅ Version {v} validated")

✅ Version 1 validated
✅ Version 2 validated
✅ Version 3 validated
✅ Version 4 validated
✅ Version 5 validated
✅ Version 6 validated
✅ Version 7 validated
✅ Version 8 validated


In [83]:
from pathlib import Path

file_path = PROJECT_FOLDER / "snapshots" / "v5.parquet"

if file_path.exists():
    file_path.unlink()
    print("✅ File deleted")
else:
    print("⚠️ File does not exist")


⚠️ File does not exist


## Snapshot Demo

In [88]:
%%time
reconstruct_version(8)

CPU times: user 31.3 s, sys: 9.51 s, total: 40.8 s
Wall time: 40.4 s


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,_row_id,trip_duration
0,2,2025-01-19 21:27:41,2025-01-19 21:37:42,1.0,1.18,1.0,N,230,161,1,...,0.5,3.15,0.0,1.0,18.90,2.5,0.0,0.75,ad58fff8e8054647bb0ac6290c4c9cf0,0 days 00:10:01
1,2,2025-01-19 21:42:22,2025-01-19 21:52:29,1.0,2.44,1.0,N,162,238,1,...,0.5,1.00,0.0,1.0,20.25,2.5,0.0,0.75,2176de14428b4ce8b3a529b339285d93,0 days 00:10:07
2,2,2025-01-19 21:07:47,2025-01-19 21:17:30,1.0,1.95,1.0,N,140,170,1,...,0.5,3.57,0.0,1.0,21.42,2.5,0.0,0.75,846c0be1c55543c793fdbe78dea586d4,0 days 00:09:43
3,2,2025-01-19 21:30:28,2025-01-19 21:33:48,4.0,0.65,1.0,N,141,262,1,...,0.5,1.08,0.0,1.0,11.88,2.5,0.0,0.00,de9e034fe3854830ba383b1dbb7aa9b3,0 days 00:03:20
4,2,2025-01-19 21:47:05,2025-01-19 21:56:20,2.0,2.68,1.0,N,141,107,1,...,0.5,4.64,0.0,1.0,23.19,2.5,0.0,0.75,b2a8f6ad08b24b0b83cd879656ce0987,0 days 00:09:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361140,2,2025-02-01 00:50:42,2025-02-01 01:07:19,1.0,2.74,1.0,N,158,233,1,...,0.5,4.69,0.0,1.0,28.14,2.5,0.0,0.75,5a68690630044361b4d77a551367def1,0 days 00:16:37
1361141,2,2025-02-01 00:25:35,2025-02-01 00:34:18,1.0,1.70,1.0,N,163,236,1,...,0.5,3.43,0.0,1.0,20.58,2.5,0.0,0.75,6baaccfd5c0045328e829019b166fbe6,0 days 00:08:43
1361142,2,2025-02-01 00:15:12,2025-02-01 00:30:44,1.0,3.54,1.0,N,249,229,1,...,0.5,4.83,0.0,1.0,28.98,2.5,0.0,0.75,c0dc18ccad7f4d728245116d15c20d84,0 days 00:15:32
1361143,2,2025-02-01 00:07:05,2025-02-01 00:21:34,1.0,2.47,1.0,N,148,66,1,...,0.5,4.27,0.0,1.0,25.62,2.5,0.0,0.75,1e18504602184edda06e5c3ffdaac7a0,0 days 00:14:29


In [89]:
commit_version(PROJECT_FOLDER, version = 5, parent = 4, snapshot_df = df_v5)

✅ Committed v5 (parent v4) with files: 


In [90]:
%%time
reconstruct_version(8)

CPU times: user 9.19 s, sys: 2.6 s, total: 11.8 s
Wall time: 11.6 s


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,_row_id,trip_duration
0,2,2025-01-19 21:27:41,2025-01-19 21:37:42,1.0,1.18,1.0,N,230,161,1,...,0.5,3.15,0.0,1.0,18.90,2.5,0.0,0.75,ad58fff8e8054647bb0ac6290c4c9cf0,0 days 00:10:01
1,2,2025-01-19 21:42:22,2025-01-19 21:52:29,1.0,2.44,1.0,N,162,238,1,...,0.5,1.00,0.0,1.0,20.25,2.5,0.0,0.75,2176de14428b4ce8b3a529b339285d93,0 days 00:10:07
2,2,2025-01-19 21:07:47,2025-01-19 21:17:30,1.0,1.95,1.0,N,140,170,1,...,0.5,3.57,0.0,1.0,21.42,2.5,0.0,0.75,846c0be1c55543c793fdbe78dea586d4,0 days 00:09:43
3,2,2025-01-19 21:30:28,2025-01-19 21:33:48,4.0,0.65,1.0,N,141,262,1,...,0.5,1.08,0.0,1.0,11.88,2.5,0.0,0.00,de9e034fe3854830ba383b1dbb7aa9b3,0 days 00:03:20
4,2,2025-01-19 21:47:05,2025-01-19 21:56:20,2.0,2.68,1.0,N,141,107,1,...,0.5,4.64,0.0,1.0,23.19,2.5,0.0,0.75,b2a8f6ad08b24b0b83cd879656ce0987,0 days 00:09:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361140,2,2025-02-01 00:26:45,2025-02-01 00:32:21,1.0,0.77,1.0,N,151,24,1,...,0.5,1.94,0.0,1.0,11.64,0.0,0.0,0.00,58de771f69404f5c8a92e6a9452b9dc3,0 days 00:05:36
1361141,2,2025-02-01 00:09:21,2025-02-01 00:25:31,1.0,1.82,1.0,N,79,231,1,...,0.5,4.27,0.0,1.0,25.62,2.5,0.0,0.75,d8272b95f0184cefa4109910f0737013,0 days 00:16:10
1361142,2,2025-02-01 00:26:14,2025-02-01 00:35:32,1.0,1.98,1.0,N,231,87,1,...,0.5,2.20,0.0,1.0,20.05,2.5,0.0,0.75,89d77fed31204804b4e2499c663b4ff0,0 days 00:09:18
1361143,2,2025-02-01 00:48:47,2025-02-01 00:49:51,1.0,0.16,1.0,N,87,209,4,...,-0.5,0.00,0.0,-1.0,-9.45,-2.5,0.0,-0.75,95ab638846b8412fb44c8fb9c0115b61,0 days 00:01:04


In [91]:
from pathlib import Path

file_path = PROJECT_FOLDER / "snapshots" / "v5.parquet"

if file_path.exists():
    file_path.unlink()
    print("✅ File deleted")
else:
    print("⚠️ File does not exist")


✅ File deleted
